In [164]:
import pandas as pd
import re

# Cleaning the NPI data to only contain NPI code and Taxonomy code

In [3]:
npi_data = pd.read_csv('data/NPPES_Data_Dissemination_February_2020/npidata_pfile_20050523-20200209.csv',
                      usecols=['NPI',
                               'Healthcare Provider Primary Taxonomy Switch_1','Healthcare Provider Primary Taxonomy Switch_2',
                              'Healthcare Provider Primary Taxonomy Switch_3','Healthcare Provider Primary Taxonomy Switch_4',
                              'Healthcare Provider Primary Taxonomy Switch_5','Healthcare Provider Primary Taxonomy Switch_6',
                              'Healthcare Provider Primary Taxonomy Switch_7','Healthcare Provider Primary Taxonomy Switch_8',
                              'Healthcare Provider Primary Taxonomy Switch_9','Healthcare Provider Primary Taxonomy Switch_10',
                              'Healthcare Provider Primary Taxonomy Switch_11','Healthcare Provider Primary Taxonomy Switch_12',
                              'Healthcare Provider Primary Taxonomy Switch_13','Healthcare Provider Primary Taxonomy Switch_14',
                              'Healthcare Provider Primary Taxonomy Switch_15','Healthcare Provider Taxonomy Code_1',
                              'Healthcare Provider Taxonomy Code_2','Healthcare Provider Taxonomy Code_3',
                              'Healthcare Provider Taxonomy Code_4','Healthcare Provider Taxonomy Code_5',
                              'Healthcare Provider Taxonomy Code_6','Healthcare Provider Taxonomy Code_7',
                              'Healthcare Provider Taxonomy Code_8','Healthcare Provider Taxonomy Code_9',
                              'Healthcare Provider Taxonomy Code_10','Healthcare Provider Taxonomy Code_11',
                              'Healthcare Provider Taxonomy Code_12','Healthcare Provider Taxonomy Code_13',
                              'Healthcare Provider Taxonomy Code_14','Healthcare Provider Taxonomy Code_15'])

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (67,70,71,74,75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Trying to define a function to return the Primary Taxonomy Code
def tax_code(data):
    if data['Healthcare Provider Primary Taxonomy Switch_1'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_1']
    elif data['Healthcare Provider Primary Taxonomy Switch_2'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_2']
    elif data['Healthcare Provider Primary Taxonomy Switch_3'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_3']
    elif data['Healthcare Provider Primary Taxonomy Switch_4'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_4']
    elif data['Healthcare Provider Primary Taxonomy Switch_5'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_5']
    elif data['Healthcare Provider Primary Taxonomy Switch_6'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_6']
    elif data['Healthcare Provider Primary Taxonomy Switch_7'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_7']
    elif data['Healthcare Provider Primary Taxonomy Switch_8'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_8']
    elif data['Healthcare Provider Primary Taxonomy Switch_9'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_9']
    elif data['Healthcare Provider Primary Taxonomy Switch_10'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_10']
    elif data['Healthcare Provider Primary Taxonomy Switch_11'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_11']
    elif data['Healthcare Provider Primary Taxonomy Switch_12'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_12']
    elif data['Healthcare Provider Primary Taxonomy Switch_13'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_13']
    elif data['Healthcare Provider Primary Taxonomy Switch_14'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_14']
    elif data['Healthcare Provider Primary Taxonomy Switch_15'] == 'Y':
        return data['Healthcare Provider Taxonomy Code_15']
    else:
        return 'none'

In [6]:
# Running the function to determine the primary taxonomy code for each npi
npi_data['taxonomy_code'] = npi_data.apply(tax_code, axis=1)

In [8]:
# Selecting only the NPI and taxonomy codes
npi_data = npi_data.iloc[:,[0,31]]

In [11]:
# Exporting the NPI/taxonomy code cleaned data for further use
npi_data.to_csv('data/npi_taxonomy_clean.csv', index=False)

# Cleaning the NPI data to contain only NPI code and entity type code

In [15]:
npi_data = pd.read_csv('data/NPPES_Data_Dissemination_February_2020/npidata_pfile_20050523-20200209.csv',
                      usecols=['NPI','Entity Type Code'])

In [18]:
# Exporting the cleaned NPI/entity code data for further use
npi_data.to_csv('data/npi_entitycode_clean.csv', index=False)

In [19]:
npi_data.head()

,NPI,Entity Type Code
0,1679576722,1.0
1,1588667638,1.0
2,1497758544,2.0
3,1306849450,1.0
4,1215930367,1.0


# Cleaning the Taxonomy Codes and Descriptions

In [119]:
taxonomy = pd.read_csv('data/CROSSWALK_MEDICARE_PROVIDER_SUPPLIER_to_HEALTHCARE_PROVIDER_TAXONOMY.csv', nrows=507)

In [120]:
# Removing all taxonomy codes that are not "Allopathic and Osteopathic Physicians"
taxonomy = taxonomy[taxonomy['PROVIDER TAXONOMY DESCRIPTION'].str.contains('Allopathic & Osteopathic Physicians', na=False)]

In [121]:
# Creating a function to split out the specialty from the taxonomy description
def specialty_split(tax_desc):
    if 'Internal Medicine' in tax_desc.split('/')[1]:
        if ',' not in tax_desc:
            return tax_desc.split('/')[1]
        return tax_desc.split(', ')[1]
    elif tax_desc.split('/')[0] == 'Allopathic & Osteopathic Physicians':
        return tax_desc.split('/')[1]
    else:
        return tax_desc.split('/')[0]

In [125]:
# Apply the function to the dataframe
taxonomy['specialty'] = taxonomy['PROVIDER TAXONOMY DESCRIPTION'].apply(specialty_split)

# Filter down to just the columns of interest (taxonomy code and specialty)
taxonomy = taxonomy[['PROVIDER TAXONOMY CODE','specialty']]

In [127]:
taxonomy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265 entries, 0 to 505
Data columns (total 2 columns):
PROVIDER TAXONOMY CODE    265 non-null object
specialty                 265 non-null object
dtypes: object(2)
memory usage: 6.2+ KB


In [129]:
# Save the cleaned Taxonomy code data to csv
taxonomy.to_csv('data/taxonomy_code_specialty_clean.csv', index=False)

# Merging the NPI and Taxonomy code description on taxonomy code

In [45]:
# Pull in NPI and taxonomy code data
npi_taxonomy = pd.read_csv('data/npi_taxonomy_clean.csv')

In [46]:
# Pull in the taxonomy code and taxonomy description data
taxonomy_desc = pd.read_csv('data/taxonomy_code_specialty_clean.csv')

In [47]:
# Merge the NPI and taxonomy description on the taxonomy code. Inner merge to only get codes for allo/osteopathic npi's
npi_taxonomy = npi_taxonomy.merge(taxonomy_desc, left_on='taxonomy_code', right_on='PROVIDER TAXONOMY CODE')

In [48]:
npi_taxonomy.head()

,NPI,taxonomy_code,PROVIDER TAXONOMY CODE,specialty
0,1679576722,207X00000X,207X00000X,Orthopedic Surgery
1,1679576722,207X00000X,207X00000X,Orthopedic Surgery
2,1205839354,207X00000X,207X00000X,Orthopedic Surgery
3,1205839354,207X00000X,207X00000X,Orthopedic Surgery
4,1578566527,207X00000X,207X00000X,Orthopedic Surgery


In [49]:
npi_taxonomy = npi_taxonomy[['NPI', 'specialty']]

In [50]:
npi_taxonomy.to_csv('data/npi_taxonomy_description.csv', index=False)

# Merging the NPI and specialty (taxonomy description) using the cleaned HOP_teaming data

In [269]:
# Import the NPI and taxonomy description data
npi_taxonomy = pd.read_csv('data/npi_taxonomy_description.csv')

In [270]:
# Import the cleaned and filtered hop_teaming data
hop_npi = pd.read_pickle('data/hop_npi.csv')

In [271]:
hop_npi.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait
4,1033194220,1003000308,62,80,26.175
5,1003818055,1003000563,101,146,0.000
13,1013967371,1003001538,46,61,21.033
14,1023073459,1003001553,25,51,34.314
15,1023003142,1003001553,164,222,33.923


In [272]:
# Merge the taxonomy description to the from_npi provider based on the npi_code
hop_npi = hop_npi.merge(npi_taxonomy, left_on='from_npi', right_on='NPI')
# Drop the unneeded column that was joined on
hop_npi = hop_npi.drop(columns='NPI')
# Rename the specialty/taxonomy description column to show it is the "from" provider
hop_npi = hop_npi.rename(columns={'specialty':'from_specialty'})

In [274]:
# Export the new data as a csv
hop_npi.to_csv('data/hop_npi_taxonomy_clean.csv', index = False)

# Merging the NPI and Zip code to the CBSA Region

In [341]:
# Import the data containing the NPI code and postal code (which contains the 5 digit zip code)
npi_data = pd.read_csv('data/NPPES_Data_Dissemination_February_2020/npidata_pfile_20050523-20200209.csv',
                      usecols=['NPI','Provider Business Practice Location Address Postal Code'])

npi_data = npi_data.rename(columns={'Provider Business Practice Location Address Postal Code':'zip_code'})

In [342]:
# Cleaning up the zip code column to remove all junk for stripping out the 5 digit zip code
npi_data = npi_data.dropna()
npi_data['zip_code'] = npi_data['zip_code'].astype(str)
npi_data['zip_code'] = npi_data['zip_code'].str.replace(' ','')
npi_data['zip_code'] = npi_data['zip_code'].str.replace('-','')
# Remove all the alphabet characters from the zip code column
npi_data['zip_code'] = npi_data['zip_code'].apply(lambda x: re.sub(r'[a-zA-Z]','', x))
# Remove trailing ".0" that is found on majority of entries
npi_data['zip_code'] = npi_data['zip_code'].str.replace('\.0','')

In [343]:
# This function takes the input (zip code column) and determines how many initial 0's were removed on importing and
# then determining how many numbers to extract to receive the proper 5 digit zip code. Also identify junk values.
def zip_finder(zippies):
    if len(zippies)==9:
        return zippies[:5]
    if len(zippies)==8:
        return '0'+zippies[:4]
    if len(zippies)==7:
        return '00'+zippies[:3]
    if len(zippies)==5:
        return zippies
    if len(zippies)==4:
        return '0'+zippies
    if len(zippies)==3:
        return '00'+zippies
    else:
        return 'no real zip'

In [344]:
# Perform the zip code cleaning function (zip_finder) and drop all values that are junk ('no real zip')
npi_data['real_zip_code'] = npi_data.zip_code.apply(zip_finder)
npi_data = npi_data[npi_data.real_zip_code!='no real zip']

In [345]:
# Remove the extra column to keep only the needed two, NPI code and zip code
npi_data = npi_data[['NPI','real_zip_code']]

In [346]:
# Import the data that contains the CBSA region codes and 5 digit zip code
cbsa_zips = pd.read_excel('data/ZIP_CBSA_122019.xlsx', usecols = ['ZIP', 'CBSA', 'TOT_RATIO'], dtype={'ZIP':object})

In [347]:
# Because some Zip codes fall into several CBSA regions, want to remove duplicates and keep the regions that cover
# the majority of the zip code.
cbsa_zips = cbsa_zips.sort_values(['ZIP', 'TOT_RATIO'], ascending = [False, False])

cbsa_zips = cbsa_zips.drop_duplicates(subset='ZIP', keep='first')

cbsa_zips = cbsa_zips.drop(columns = 'TOT_RATIO')

In [348]:
# Merge the data containing the NPI code and CBSA region code based upon the zip code values
npi_data = npi_data.merge(cbsa_zips, left_on='real_zip_code', right_on='ZIP', how='left')

npi_data = npi_data[['NPI','CBSA']]

In [349]:
# Export the cleaned data containing NPI code and CBSA region code as a csv file
npi_data.to_csv('data/npi_cbsa_clean.csv', index = False)

# Importing the Provider Organization name and NPI

In [283]:
npi_data = pd.read_csv('data/NPPES_Data_Dissemination_February_2020/npidata_pfile_20050523-20200209.csv',
                      usecols=['NPI','Provider Other Organization Name'])

npi_data = npi_data.rename(columns={'Provider Other Organization Name':'to_organization'})

In [285]:
# Export the data containing NPI code and provider organization name for entity type 2 as a csv file
npi_data.to_csv('data/entity2_org_name.csv', index = False)

# Merging the CBSA region code, organization/provider name, and provider state to the full cleaned NPI hop data

In [403]:
# Importing the hop npi data, cbsa region, and provider organizatin name data for merging together
npi_data = pd.read_csv('data/hop_npi_taxonomy_clean.csv')
cbsa_regions = pd.read_csv('data/npi_cbsa_clean.csv')
provider_to_name = pd.read_csv('data/entity2_org_name.csv')

In [404]:
# Merge all 3 data sets together into a single dataframe
npi_data = npi_data.merge(cbsa_regions, left_on = 'from_npi', right_on = 'NPI', how='left').merge(provider_to_name,
                                                                                                 left_on='to_npi',
                                                                                                 right_on='NPI')
# Drop the duplicate NPI code columns
npi_data = npi_data.drop(columns=['NPI_x', 'NPI_y'])

In [405]:
npi_data.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,from_specialty,CBSA,to_organization
0,1033194220,1003000308,62,80,26.175,"Radiology, Diagnostic Radiology",12060.0,"BUCKHEAD EAR, NOSE AND THROAT"
1,1295734820,1003000308,46,52,49.462,Cardiovascular Disease,12060.0,"BUCKHEAD EAR, NOSE AND THROAT"
2,1295734820,1003000308,46,52,49.462,Cardiovascular Disease,12060.0,"BUCKHEAD EAR, NOSE AND THROAT"
3,1518973437,1003000308,44,51,33.843,Cardiovascular Disease,12060.0,"BUCKHEAD EAR, NOSE AND THROAT"
4,1518973437,1003000308,44,51,33.843,Cardiovascular Disease,12060.0,"BUCKHEAD EAR, NOSE AND THROAT"


In [406]:
# Importing the NPI code and provider last and first name to create a single name column to merge to full dataset above
npi_from_name = pd.read_csv('data/NPPES_Data_Dissemination_February_2020/npidata_pfile_20050523-20200209.csv',
                      usecols=['NPI','Provider Last Name (Legal Name)', 'Provider First Name'])

In [407]:
# Adding a column for provider name by combining provider first and last name columns and then dropping those
npi_from_name['from_provider_name'] = npi_from_name['Provider Last Name (Legal Name)'] +', ' + npi_from_name['Provider First Name']

npi_from_name = npi_from_name.drop(columns=['Provider Last Name (Legal Name)', 'Provider First Name'])

In [408]:
# Merge the full dataset to the from_provider_name data
npi_data = npi_data.merge(npi_from_name, left_on = 'from_npi', right_on = 'NPI', how='left')

# Drop the duplicate NPI code column after the merge
npi_data = npi_data.drop(columns='NPI')

In [409]:
# Importing the NPI code and provider practice location state name to merge with full dataset
npi_state = pd.read_csv('data/NPPES_Data_Dissemination_February_2020/npidata_pfile_20050523-20200209.csv',
                      usecols=['NPI','Provider Business Practice Location Address State Name'])


npi_state = npi_state.rename(columns={'Provider Business Practice Location Address State Name':'from_state'})

In [410]:
npi_data = npi_data.merge(npi_state, left_on = 'from_npi', right_on = 'NPI', how = 'left')
npi_data = npi_data.drop(columns='NPI')

In [413]:
# Export the final cleaned data as a csv for uploading
npi_data.to_csv('data/cleaned_npi_taxonomy_names_cbsa.csv', index=False)

In [414]:
# Export the final cleaned data as a pickle for faster uploading
npi_data.to_pickle('data/cleaned_npi_taxonomy_names_cbsa_pickle.csv')

In [411]:
npi_data.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,from_specialty,CBSA,to_organization,from_provider_name,from_state
0,1033194220,1003000308,62,80,26.175,"Radiology, Diagnostic Radiology",12060.0,"BUCKHEAD EAR, NOSE AND THROAT","JACOBS, LOUIS",GA
1,1295734820,1003000308,46,52,49.462,Cardiovascular Disease,12060.0,"BUCKHEAD EAR, NOSE AND THROAT","ARNOLD, WILLIAM",GA
2,1295734820,1003000308,46,52,49.462,Cardiovascular Disease,12060.0,"BUCKHEAD EAR, NOSE AND THROAT","ARNOLD, WILLIAM",GA
3,1518973437,1003000308,44,51,33.843,Cardiovascular Disease,12060.0,"BUCKHEAD EAR, NOSE AND THROAT","COOK, ARTHUR",GA
4,1518973437,1003000308,44,51,33.843,Cardiovascular Disease,12060.0,"BUCKHEAD EAR, NOSE AND THROAT","COOK, ARTHUR",GA


In [412]:
npi_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9175634 entries, 0 to 9175633
Data columns (total 10 columns):
from_npi              int64
to_npi                int64
patient_count         int64
transaction_count     int64
average_day_wait      float64
from_specialty        object
CBSA                  float64
to_organization       object
from_provider_name    object
from_state            object
dtypes: float64(2), int64(4), object(4)
memory usage: 770.0+ MB
